# Batch processing strategies: vertical and horizontal integration

Congratulations, you've just completed an awesome image processing pipeline that takes an image and does something rather useful. You've tested it a few times and you're happy with the results. Now you need to apply the same operation to 1000 images. Maybe you need to apply them to 20 datasets with slightly different parameters. Maybe you need to aggreagate information across the results of each processed image to make your conclusions. Doing this can be hard, but sometimes it's so easy you can see many ways to approach this, and are not sure which one is the best. Let's talk about what you need to keep in mind when building systems to actually apply your pipeline.

In [41]:
%matplotlib inline

**Exercise** You're at the point where you have the functions below. How might you apply the pipeline you developed to every image in a folder, saving the results in a list?

In [42]:
from skimage.io import imread

def loading(image_file_name):
    return imread(image_file_name)

def preprocessing(image):
    return image # real code would go here

def info_extraction(image):
    results = None
    return results

data_folder = "../data"

In [43]:
import os
os.chdir(data_folder)

files = [f for f in listdir(data_folder) if '.tif' in f]
results = []

for f in files:
    img = loading(f)
    processed = preprocessing(img)
    result = info_extraction(processed)
    results.append(result)

What made this a pretty straightforward proceedure is that we encapsulated _everything_ in a function. If we developed our pipeline without doing this, we could wrap everthing in a giant function which took as parameters everything we needed.

In [44]:
def whole_pipeline(image_file_name):
    pass # code would go here

This may not be the best idea. Maybe next week you want to load similar images the same way, preprocess them differently, then extract the same information. You could have imported the `loading` and `info_extraction` you already have.

You may also be thinking of an alternative way of solving this problem that looks like this.

In [45]:
files = [f for f in listdir(data_folder) if '.tif' in f]
images = []
processed = []
results = []

for f in files:
    images.append(loading(f))
    
for img in images:
    processed.append(preprocessing(img))
    
for proc in processed:
    results.append(info_extraction(proc))

Instead of applying all processing steps to each image one at a time, you can apply each processing step to each image. So now rather than completing your pipline "vertically", from start to finish, top to bottom, you are completing your pipeline "horizontally", applying each stage across the board in sequence.

TODO: graphics of horizontal and vertical strategies

In fact, when you've made your code modular with little functions, Python tries to support this approach with _functional programming tools_. We'll talk about one of the especially useful ones:

- `map(function, iterable)`. Applies `function` to each element in `iterable` (i.e. anything you can loop through) and gives you a new list with the results.

In [46]:
files = [f for f in listdir(data_folder) if '.tif' in f]
images = map(loading, files)
processed = map(preprocessing,images)
results = map(info_extraction, processed)

Now this looks like a pretty concise way to process a pipeline! But is it the best way?

One thing to consider is that at the end of the program, `files`, `images`, `processed`, and `results` are all full arrays available to you in program memmory. This could be useful for debugging (did everything look good after preprocessing?). It may also be useful because you need to aggregated across intermediate results. Maybe you only want to use images where the background level is $\pm 10\%$ of the median across all images in your experiment (if Otsu's method sets the background threshold very high or low, it's suspicious).

The downside of having all the intermediate information is that it may use a lot of memory.

**Exercise** Not all lines above are as memory intensive as the others. Which line is the least offensive?

**Exercise** in our lesson on loading an image we discussed how much memory an image can use up. If you are processing 1000 16-bit, 1024x1024 pixel images each with 3 channels, how much memory does the first line above use?

In [47]:
num_bytes = 1000 * 2 * 1024**2 * 3
print("{} Bytes".format(num_bytes))
print("{} GB".format(num_bytes / 1024.**3))

That's probably not going to go smoothly if you have a multi-stage pipeline. You could delete each stage as you go, keeping in memory only the last and current stage using `del` stagements, e.g. `del images`, but this can introduce bugs and still requires you to keep copies of everything. For very large numbers of images, this strategy suffers.

A good way to design a pipeline is to draw out the pipes. Map out the dependencies between pieces of information so you can see where to break things into functions and figure out how much data you actually need to keep in memory. The example below is a workflow for processing a red structural channel and a green calcium fluorescence channel in a 60-frame time series of images. The goal is to find cells, make masks, and track the fluorescence intensities over time.

![Diagram of a pipeline](../fig/pipeline_diagrams.png "A typical pipeline diagram")

What are some observations we can make once we've drawn out this plan? We can see that loading raw images and preprocessing them can be done with one function, applied image-by-image, without saving the intermediate. However we need to stop after this and max project. We can then remove all of the green channel data from memory and work on the red channel, keeping only the cell location masks. We then apply the cell location masks to each image, one at a time, take out resulting total cell intensities and save them in an array.

Without doing this we may have tried to load both channels at once. This approach reduces our memory consumption by 50%.